### exception test

In [3]:
import os
import ipyparallel as ipp

ipyclient = ipp.Client()
print ipyclient.ids

[0, 1, 2, 3]


### create different view of Engines


In [53]:
#### all 4 engines
dview = ipyclient[:]

#### auto-assign across 4 Engines
lview = ipyclient.load_balanced_view()

In [ ]:

### Boss class to balance work among Node classes
class Boss():
    


In [119]:
import time
import ipyparallel as ipp

#### client class holding threaded views
class Worker():
    """ independent threaded CPUs"""
    def __init__(self, name, ipyclient, targets, func, data):
        ## Engine views
        self.client = ipyclient
        self.lview = self.client.load_balanced_view(targets=targets)
        self.dview = self.client.direct_view(targets=targets)
        self.ids = targets
        self.name = name
        
        ## how long to wait between checking for available proc
        self.delay = 0.5
        
        ## store the data and func
        self.data = data
        self.func = func
        
        ## is job finished
        self.result = {}
        
        ## finished results
        self.running = 0
        self.finished = 0
        
    
    def synchronize(self):
        """ checks whether it's finished """
        try:
            self.result[self.name] = self.running.get(0)
        except ipp.error.TimeoutError:
            pass
        except Exception as e:
            self.finished = 1
            #print(data, ': fail', e.traceback)
            self.result[self.name] = e
        else:
            self.finished = 1
            self.running = None
            

    def job(self, data, func):
        """ while job is running store async result in running."""
        self.running = self.lview.apply_async(func, data)
        
                
    def run(self):
        ## starts self.running
        self.job(self.data, self.func)
        while self.running:
            print(self.name, 'pending...')
            ## check for jobs finished, stops self.running
            self.synchronize()
            time.sleep(self.delay)

In [ ]:
class Boss():
    """ a scheduler for threaded Workers tasks """
    def __init__(self, workers):
        
        

In [ ]:
rc = ipp.Client(#'/home/user/.starcluster/ipcluster/mycluster-us-east-1.json'
                sshkey='/home/user/.ssh/mykey.rsa',
                sshserver="deren@omega.hpc.yale.edu",
                password="")

In [124]:
## controller file is located on your LOCAL machine
contro_file = "/home/deren/.ipython/profile_default/security/ipcontroller-client.json"
sshkey_file = "/home/deren/.ssh/id_rsa"

## open a connection so that all files will be found and made 
## on the REMOTE machine
ipyclient = ipp.Client(contro_file, sshkey_file)

## Now tell our data object that it's executing remotely
## data1.ipcluster = ipyclient

In [127]:
ipyclient.ids
dview = ipyclient[:]
import socket
hosts = dview.apply_sync(socket.gethostname)
print hosts

['oud', 'oud', 'oud', 'oud', 'oud', 'oud', 'oud', 'oud', 'oud', 'oud']


In [120]:
## create two Workers 
node1 = Worker("oud_1", ipyclient, [0,1], func=sum, data=range(3))
node2 = Worker("oud_2", ipyclient, [2,3], func=sum, data=range(3))

## schedule Workers jobs w/ Boss
scheduler = Boss(nodes=[node,node2])
for job in jobs:
    scheduler.add(job)

node1.run()
print node1.result

NameError: name 'Boss' is not defined

In [38]:
        
threaded1 = ipyclient.direct_view(targets=[0,1])
threaded2 = ipyclient.direct_view(targets=[2,3])
threads = [threaded1, threaded2]

In [21]:
import time
## sleep for 1 second
time.sleep(1)
## data to sleep for x seconds (0-7)
data = range(8)

### Time how long each client takes to go through sleep data

In [25]:
%%timeit
## dview map
job = dview.map(time.sleep, data)
job.get()

1 loops, best of 3: 13.1 s per loop


In [30]:
%%timeit
## dview map_async
job = dview.map_sync(time.sleep, data)
#job.get()

## dview
#[i.get() for i in job]

1 loops, best of 3: 13.1 s per loop


In [32]:
job = dview.map_async(time.sleep, data)
job.wait_interactive()

   4/4 tasks finished after   13 s
done


### Load balanced does it faster than direct
Since direct divvies up the jobs across processors before running them, whereas load_balanced assigns them as you go depending on who finished first. 

In [33]:
job = lview.map_async(time.sleep, data)
job.wait_interactive()

   8/8 tasks finished after   10 s
done


In [36]:
%%timeit 
job = lview.map_sync(time.sleep, data)
job

1 loops, best of 3: 10.1 s per loop


### Set up a custom threaded job

In [52]:
dview.scatter

defaultdict(dict,
            {u'03db1f18-0e7f-4c47-9aba-16b42eca03fe': [None],
             u'0411f754-2976-462d-96f1-8810eb4d902b': [None, None],
             u'045b2470-f4e1-44f4-9c99-27db5360bcde': [None],
             u'0486fd50-2c67-44e2-995a-c14812f5e3b8': [None, None],
             u'069deea9-952c-4dda-b7cf-b605f2fe9ae8': [None, None],
             u'074951d1-7ada-451b-a53a-36d61a6e8100': [None],
             u'09d47326-ed81-4fb7-84d5-5ce6dcf86f0d': [None],
             u'0dc9c888-70f5-44a0-a35a-e37f37b784c5': [None],
             u'0e0a94fe-1c8a-4cef-b1ce-6f01ef6bb5ec': [None, None],
             u'0f20778b-6835-4815-97a1-cea9b9f15c76': [None, None],
             u'0f7498a1-57dc-4814-90c7-c17de40fcef2': [None, None],
             u'0fb9179b-c505-482a-ae73-ddefdd392c79': [None],
             u'10518f72-2a7f-4f65-986b-171f38c28e0d': ipyparallel.error.RemoteError(u'TypeError',
                                           u'a float is required'),
             u'10606a2e-5513-4331-84

In [47]:
data.pop(0)

1

In [50]:
data = range(8)

while data:
    dat = data.pop(0)
    print dat
    for node in threads:
        try:
            ## find an available engine
        
    

0
1
2
3
4
5
6
7


In [93]:
## make data 
data = range(20)

## make function that will raise exception for one data point
@lbview.parallel(block=False)
def testfunc(x):
    import os
    if x[0] < 10: 
        raise Exception("fail")
    else:
        return (x, os.getpid())
    

In [95]:
res = testfunc([3, 30, 40, 3, 4])


In [96]:
[i.get() for i in res]

CompositeError: one or more exceptions from call to method: testfunc
[1:apply]: Exception: fail
[0:apply]: Exception: fail
[3:apply]: Exception: fail

In [70]:
## now run all the data across 4 processors
results = []
try:
    res = lbview.map_async(testfunc, data)
    results = res.get()
    #    results.append(res)
    #ipyclient.wait(results)
    
except Exception:
    print 'hi'
    
finally:
    print 'done'

hi
done


In [71]:
final = [res.get() for i in results]
print len(final), final

0 []
